In [1]:
from DataParser import DataParser
import numpy as np
import scipy.sparse as sps

from Data_manager.split_functions.split_train_validation_random_holdout import \
split_train_in_two_percentage_global_sample

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from SimpleEvaluator import evaluator
from Base.Evaluation.Evaluator import EvaluatorHoldout

In [2]:
from Base.NonPersonalizedRecommender import TopPop, Random
from Hybrid.HybridCombinationSearchCVtest import HybridCombinationSearchCVtest
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from MatrixFactorization.IALSRecommender import IALSRecommender
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from SLIM_ElasticNet.SSLIM_ElasticNet import SSLIMElasticNet
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender
from MatrixFactorization.NMFRecommender import NMFRecommender

In [ ]:
seed = 1666
parser = DataParser()

URM_all = parser.get_URM_all()
ICM_all = parser.get_ICM_all()

#URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.85, seed=seed)

In [ ]:
p3b = (RP3betaRecommender, {'topK': 121, 'alpha': 0.20195468746532336, 'beta': 0.004377103282444673, 'normalize_similarity': False})
p3a = (P3alphaRecommender, {'topK': 344, 'alpha': 0.4233795425391197, 'normalize_similarity': False})
icf = (ItemKNNCFRecommender, {'topK': 994, 'shrink': 981, 'similarity': 'asymmetric', 'normalize': True, 'asymmetric_alpha': 0.05110465631417439, 'feature_weighting': 'TF-IDF'})
ucf = (UserKNNCFRecommender, {'topK': 342, 'shrink': 1, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'none'})
icb = (ItemKNNCBFRecommender, {'topK': 22, 'shrink': 59, 'similarity': 'dice', 'normalize': False})
sen = (SLIMElasticNetRecommender, {'topK': 992, 'l1_ratio': 0.004065081925341167, 'alpha': 0.003725005053334143})
sbpr = (SLIM_BPR_Cython, {'topK': 979, 'epochs': 130, 'symmetric': False, 'sgd_mode': 'adam', 'lambda_i': 0.004947329669424629, 'lambda_j': 1.1534760845071758e-05, 'learning_rate': 0.0001})
sslim = (SSLIMElasticNet, {'beta': 0.4849594591575789, 'topK': 1000, 'l1_ratio': 1e-05, 'alpha': 0.001})

In [ ]:
combo_labels ={
    'combo1': 'icb_icf_ucf',
    'combo2': 'icb_icf_rp3b',
    'combo3': 'icb_icf_p3a',
    'combo4': 'icb_ucf_rp3b',
    'combo5': 'icb_ucf_p3a',
    'combo6': 'icb_p3a_rp3b',
}

In [ ]:
kf = KFold(n_splits=k, shuffle=True, random_state=self.seed)

URM_list = []
URM_test_list = []
evaluator_list = []

shape = URM_all.shape
indptr = URM_all.indptr
indices = URM_all.indices
data = URM_all.data

for train_index, test_index in kf.split(data):
    data_train = np.ones(data.shape)
    data_test = np.ones(data.shape)
    data_train[test_index] = 0
    data_test[train_index] = 0
    kf_train = sps.csr_matrix((data_train, indices, indptr), shape=shape).copy()
    kf_test = sps.csr_matrix((data_test, indices, indptr), shape=shape).copy()
    kf_train.eliminate_zeros()
    kf_test.eliminate_zeros()
    URM_list.append(kf_train)
    URM_test_list.append(kf_test)
    evaluator_list.append(EvaluatorHoldout(kf_test, cutoff_list=[10]))

In [ ]:
list_recommender = [icb, icf, ucf]
result_list = []
for i in range(len(URM_list)):
    combo1 = HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.17669196327123107, 'l1_ratio': 0.3076285474940705} #MAP 0.0563723
    combo1.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo1)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

In [ ]:
list_recommender = [icb, icf, rp3b]
result_list = []
for i in range(len(URM_list)):
    combo2 = HybridCombinationSearchCVtest(URM_train, ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.4137511907513143, 'l1_ratio': 0.2562027806859107} #MAP 0.0553861
    combo2.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo2)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

In [ ]:
list_recommender = [icb, icf, p3a]
result_list = []
for i in range(len(URM_list)):
    combo3 = HybridCombinationSearchCVtest(URM_train, ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.6795644006301982, 'l1_ratio': 0.16897463528047377} #MAP 0.0569727
    combo3.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo3)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

In [ ]:
list_recommender = [icb, ucf, rp3b]
result_list = []
for i in range(len(URM_list)):
    combo4 = HybridCombinationSearchCVtest(URM_train, ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.22386976323299096, 'l1_ratio': 0.5121960118838208} #MAP 0.0552285
    combo4.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo4)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

In [ ]:
list_recommender = [icb, ucf, p3a]
result_list = []
for i in range(len(URM_list)):
    combo5 = HybridCombinationSearchCVtest(URM_train, ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.5989306837162718, 'l1_ratio': 0.19159245289609933} #MAP 0.0570313
    combo5.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo5)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

In [ ]:
list_recommender = [icb, rp3b, p3a]
result_list = []
for i in range(len(URM_list)):
    combo6 = HybridCombinationSearchCVtest(URM_train, ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.3003711591677414, 'l1_ratio': 0.520142870843707} #MAP 0.0568581
    combo6.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo6)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))